In [6]:
include("optim_lib.jl")
include("lib_func.jl")

# Constants and Parameters
d1, d2, ds = 3, 8, 1  # Degree range and step
const n, a, b = 3, 10, 1 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .9 , 2 / 10  # Sampling parameters
f = alpine1 # Objective function

#39 (generic function with 1 method)

In [7]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.1)

dimension Vector space: 20
sample size: 27
dimension Vector space: 35
sample size: 64
dimension Vector space: 56
sample size: 64
dimension Vector space: 84
sample size: 125
dimension Vector space: 120
sample size: 125
dimension Vector space: 165
sample size: 216


6-element Vector{Vector{Float64}}:
 [14.372461441119478, -6.818460280949621, 1.506771454196918, 2.0329204347832344e16, -4.6309397699943045, 3.666200521134394e-16, -8.855781934749717e-17, 2.203337230220576, -3.515792705025695e-16, 1.4641313141089026e16, -0.519920670674602, 1.0850748945981426e-16, 8.103752272183292e-17, 1.0813066305340314e-15, -2.0815489616447702e-16, -8.740695151656005e-17, 3.5123970515226155, -3.8227405555209935e-16, 7.562864140930623e-16, 3.9520285064216585e15]
 [2.8378738523744076, -4.220048442775064, -9.608705049052563, 12.602311746792108, -7.731345738692395e16, -0.23056468729606316, 5.50662685129793e-16, 3.1335119848465986e-16, 2.4522994236177673e-16, -2.2371002751059375  …  7.483383973022746, -1.0293254508169808e-15, 1.5579983778653017e-16, -3.764148273375741e-15, 1.7411715385196602e-16, 1.2221673325707448e-15, -9.729578761821566, -1.098926514267198e-15, -6.199425205247112e-16, 6.783374017708691e16]
 [18.50230439211035, -5.330780181533039, 0.3798029860318845, -205

In [8]:
using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, h_z, col = main_3d(n, d1, d2, ds, coeffs_poly_approx) # main_2d is in hom_solve.jl
df = DataFrame(x = C*h_x, y = C*h_y, z = C*h_z, col=col)
df[!, :result] = [f([df.x[i], df.y[i], df.z[i]]) for i in 1:nrow(df)];
println(df)


Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 2    Time: 0:00:01
  mixed_volume:  8


Tracking 8 paths...  25%|███████▊                       |  ETA: 0:00:20



Tracking 8 paths... 100%|███████████████████████████████| Time: 0:00:07
  # paths tracked:                  8
  # non-singular solutions (real):  8 (8)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         8 (8)
Degree: 3
Number of solutions: 8
Tracking 27 paths...   7%|██▎                           |  ETA: 0:00:34



Tracking 27 paths... 100%|██████████████████████████████| Time: 0:00:02
  # paths tracked:                  27
  # non-singular solutions (real):  27 (27)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         27 (27)
Degree: 4
Number of solutions: 27
Tracking 64 paths...   3%|█                             |  ETA: 0:01:37



Tracking 64 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  64
  # non-singular solutions (real):  64 (32)
  # singular endpoints (real):      0 (0)
  # total solutions (real):       

In [9]:
nrow(df)

152

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.


In [11]:
using Optim 

for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    println(res)
end

Optimizing for point 1
Iter     Function value   Gradient norm 
     0     1.488386e+01     5.593867e-01
 * time: 0.01895880699157715
     1     6.988451e+00     6.641710e+00
 * time: 0.7068870067596436
     2     5.711179e+00     6.356462e+00
 * time: 0.7071049213409424
     3     5.703411e+00     6.290382e+00
 * time: 0.7071728706359863
     4     1.678317e+00     6.535239e+00
 * time: 0.7072288990020752
     5     8.761314e-02     1.260094e+01
 * time: 0.7072908878326416
     6     6.581694e-02     1.260445e+01
 * time: 0.7073488235473633
     7     3.545759e-02     1.260360e+01
 * time: 0.7074029445648193
     8     1.677612e-02     5.168461e+00
 * time: 0.7074649333953857
     9     1.630018e-02     2.084616e+00
 * time: 0.7075188159942627
    10     1.608880e-02     9.289498e-01
 * time: 0.7075729370117188
    11     2.503007e-03     1.359445e-02
 * time: 0.7076308727264404
    12     2.503010e-03     2.521573e-02
 * time: 0.7077047824859619
    13     2.499363e-03     2.602552e-

Excessive output truncated after 524314 bytes.

Doesn't make too much sense, but maybe we could plot the level sets of the polynomial approximants